In [ ]:
import pandas as pd
from mistralai import Mistral
import time

# Initialisation du client avec la clé API
client = Mistral(api_key="QU9joTThl5C9C8Waz8dgk2Mu9mFT7TaZ")


df = pd.read_csv(r"C:\Users\Harrison\Challenge48\data\cleaned_tweets.csv")

# Fonction pour analyser le sentiment d'un tweet avec un délai
def get_sentiment(tweet):
    content = "Analyse du sentiment du tweet : " + tweet
    response = client.agents.complete(
        agent_id="ag:f685da10:20250310:untitled-agent:5d68c6b9",  # Remplace par ton ID d'agent
        messages=[{
            "role": "user",
            "content": content
        }]
    )
    # Récupérer la valeur du sentiment dans la réponse JSON
    response_text = response.choices[0].message.content.strip()
    return response_text

# Appliquer la fonction à chaque tweet avec un délai de 4 secondes entre chaque requête
sentiments = []
for tweet in df['tweet_text']:
    sentiment = get_sentiment(tweet)
    sentiments.append(sentiment)
    time.sleep(4)  # Pause de 4 secondes entre les requêtes

# Ajouter les résultats dans la nouvelle colonne "sentiment"
df['sentiment'] = sentiments

# Afficher le DataFrame avec la nouvelle colonne "sentiment"
df.head()


KeyboardInterrupt: 

In [33]:
from mistralai import Mistral

# Initialisation du client avec la clé API
client = Mistral(api_key="QU9joTThl5C9C8Waz8dgk2Mu9mFT7TaZ")

# Le contenu du tweet que tu veux analyser
content = "C'est une super journée, je suis tellement content !"

# Ajout d'un préfixe pour spécifier l'analyse de sentiment
content = "Analyse du sentiment du tweet : " + content

# Utilisation de l'agent pour obtenir la réponse
response = client.agents.complete(
    agent_id="ag:f685da10:20250310:untitled-agent:5d68c6b9",  # Remplace par ton ID d'agent
    messages=[{
        "role": "user",
        "content": content
    }]
)

# Récupérer et afficher la réponse générée par l'agent
response_text = response.choices[0].message.content.strip()
print("Sentiment analysé : ", response_text)


Sentiment analysé :  {
  "inconfort": 0,
  "sentiment": positif,
  "urgence": False
}


In [1]:
import pandas as pd

#data = pd.read_csv(r"C:\Users\Harrison\Challenge48\data\cleaned_tweets_with_sentiment.csv")
df = pd.read_csv(r"C:\Users\Harrison\Challenge48\data\cleaned_tweets_with_sentiment.csv")

df.head()

,user_id,screen_name,name,date,tweet_text,hour,tweet_length,criticite,sentiment
0,189684000000000011534336,lescausesjustes,AInnovation,2025-03-04 08:09:58+00:00,"@ENGIEgroup @ENGIEpartFR pour l'amour du ciel,...",8,113,1,Positif 😊
1,18967000000000000262144,Leoletonneau,Tigre & Dragon 🐯 🐉,2025-03-03 23:25:45+00:00,"@ENGIEpartFR vraiment des escrocs, des montant...",23,131,1,Négatif 😡
2,189662999999999998164992,SouareFirst,So First,2025-03-03 18:47:06+00:00,@ENGIEpartFR bonjour engie les voleurs ! Atten...,18,184,1,Négatif 😡
3,189661999999999987941376,julien_ducerf,Julien Ducerf,2025-03-03 17:50:42+00:00,@ENGIEpartFR c'est du harcèlement pour augment...,17,167,2,Négatif 😡
4,189658000000000014155776,djofthp,jojodamido,2025-03-03 15:24:10+00:00,@ENGIEpartFR on vous parle,15,26,1,Négatif 😡


In [ ]:
import pandas as pd
import requests

# Charger le DataFrame
df = pd.read_csv(r"chemin/vers/ton_fichier.csv")

# Filtrer les tweets négatifs
df_negatifs = df[df['sentiments'] == 'négatif'].copy()

# Configuration de l'API Mistral
API_KEY = "QU9joTThl5C9C8Waz8dgk2Mu9mFT7TaZ"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "User-Agent": "ag:f685da10:20250310:untitled-agent:5d68c6b9"
}

# Fonction pour interroger le modèle Mistral
def get_intent(tweet):
    prompt = f"""
    Analyse ce tweet et identifie la cause principale du sentiment négatif. 
    Précise si la négativité est due à un retard de livraison, un produit défectueux, 
    un mauvais service client ou une autre raison. Si tu n'es pas sûr, indique "autre".
    Réponds uniquement par la raison identifiée.

    Tweet : "{tweet}"
    """

    response = requests.post("https://console.mistral.ai/home ", json={"prompt": prompt}, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get("result", "autre")
    else:
        return "erreur_api"

# Appliquer la fonction sur les tweets négatifs
df_negatifs['intent'] = df_negatifs['tweets'].apply(get_intent)

# Fusionner avec le DataFrame original
df = df.merge(df_negatifs[['tweets', 'intent']], on='tweets', how='left')

# Sauvegarder le résultat
#df.to_csv("chemin/vers/fichier_sortie.csv", index=False)

print("Traitement terminé. Résultats enregistrés !")


SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}